In [ ]:
# Object Size Estimation System for AWS SageMaker

This notebook provides a complete object size estimation system using SAM (Segment Anything Model) and computer vision techniques. It detects yellow objects and rulers in images, then calculates real-world measurements.

## Features
- SAM-based object segmentation
- HSV color-based yellow object detection
- Sequential ruler marking detection with OCR
- Automatic calibration and measurement
- Enhanced visualization with annotations


In [ ]:
## 1. Install Dependencies

### Important Notes for AWS SageMaker:

**NumPy 2.x Compatibility Issue Fix:**
AWS SageMaker often comes with NumPy 2.x pre-installed. If you encounter errors like:
- `numpy 1.x cannot be run in numpy 2.2.6`
- `numpy.core.multiarray failed to import`
- `dtype size changed` errors

This happens when packages compiled for NumPy 1.x try to run with NumPy 2.x.

**Solution Steps:**
1. **DO NOT** downgrade NumPy - keep the existing NumPy 2.x
2. Install packages that are compatible with NumPy 2.x (newer versions)
3. Restart kernel if you encounter import issues
4. Use alternative installation methods if problems persist

**Key Changes:**
- Uses NumPy 2.x compatible packages (OpenCV ≥4.9, Matplotlib ≥3.8, etc.)
- PyTorch ≥2.1.0 with NumPy 2.x support
- Latest EasyOCR version (≥1.7.1)

**AWS SageMaker Specific:**
- Uses CPU-only PyTorch for compatibility
- All models download automatically to `/tmp/models/`
- Processing works on uploaded images in `/tmp/data/`


In [ ]:
# Install required packages compatible with NumPy 2.x (AWS SageMaker default)
# Fix compatibility issues by using packages that support NumPy 2.x
!pip install --upgrade pip

# Check current NumPy version
import numpy as np
print(f"Current NumPy version: {np.__version__}")

# Install packages compatible with NumPy 2.x
!pip install --upgrade numpy  # Keep the existing NumPy 2.x
!pip install opencv-python>=4.9.0  # Supports NumPy 2.x
!pip install matplotlib>=3.8.0  # Supports NumPy 2.x  
!pip install Pillow>=10.0.0  # Latest version

# Install PyTorch with NumPy 2.x support
!pip install torch>=2.1.0 torchvision>=0.16.0 --index-url https://download.pytorch.org/whl/cpu

# Install SAM
!pip install git+https://github.com/facebookresearch/segment-anything.git

# Install EasyOCR latest version (supports NumPy 2.x)
!pip install easyocr>=1.7.1

print("✅ All dependencies installed with NumPy 2.x compatibility!")
print("Note: Restart kernel if you encounter import issues.")


In [ ]:
### Alternative Installation (if above fails)

If you still encounter NumPy compatibility issues, try this alternative approach:


In [ ]:
# Alternative installation methods for persistent NumPy 2.x compatibility issues
# Run this cell ONLY if the previous installation failed

# Method 1: Clean reinstall of all packages
# !pip uninstall -y opencv-python matplotlib torch torchvision easyocr
# !pip install --no-cache-dir opencv-python>=4.9.0 matplotlib>=3.8.0 
# !pip install --no-cache-dir torch>=2.1.0 torchvision>=0.16.0 --index-url https://download.pytorch.org/whl/cpu
# !pip install --no-cache-dir easyocr>=1.7.1

# Method 2: Force upgrade all packages
# !pip install --upgrade --force-reinstall numpy opencv-python matplotlib torch torchvision easyocr

# Method 3: Use conda for core packages (if available)
# !conda install -c conda-forge numpy opencv matplotlib -y
# !pip install torch torchvision easyocr

# Method 4: Install specific working versions for NumPy 2.x
# !pip install numpy==2.1.0 opencv-python==4.10.0.84 matplotlib==3.9.0 easyocr==1.7.1

print("Uncomment and run one of the methods above if you encounter issues")
print("Always restart kernel after changing package versions")


In [ ]:
### Verify Installation


In [ ]:
# Verify that all packages can be imported successfully
try:
    import numpy as np
    print(f"✅ NumPy version: {np.__version__}")
    
    import cv2
    print(f"✅ OpenCV version: {cv2.__version__}")
    
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    
    import easyocr
    print(f"✅ EasyOCR imported successfully")
    
    from segment_anything import sam_model_registry, SamPredictor
    print(f"✅ SAM imported successfully")
    
    import matplotlib.pyplot as plt
    print(f"✅ Matplotlib imported successfully")
    
    from PIL import Image
    print(f"✅ PIL imported successfully")
    
    print("\n🎉 All packages imported successfully! Ready to proceed.")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please restart kernel and try the alternative installation methods above")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    print("Please restart kernel and reinstall dependencies")


In [ ]:
### Fix Matplotlib NumPy 2.x Issue

If matplotlib import fails with `numpy.core.multiarray` error, run this cell:


In [ ]:
# Fix matplotlib NumPy 2.x compatibility issue
# Run this ONLY if matplotlib import fails

try:
    import matplotlib.pyplot as plt
    print("✅ Matplotlib imports successfully")
except ImportError as e:
    print(f"❌ Matplotlib import failed: {e}")
    print("Attempting to fix...")
    
    # Force reinstall matplotlib with NumPy 2.x support
    import subprocess
    subprocess.run(["pip", "install", "--upgrade", "--force-reinstall", "matplotlib>=3.8.0"], check=True)
    
    # Also reinstall contourpy which often causes issues
    subprocess.run(["pip", "install", "--upgrade", "--force-reinstall", "contourpy"], check=True)
    
    print("✅ Matplotlib reinstalled. Please restart kernel and try again.")
    
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    print("Please restart kernel and run the main installation cell again.")


In [ ]:
## 2. Import Libraries


In [ ]:
# Import core libraries with enhanced error handling for NumPy 2.x
try:
    import os
    import re
    import urllib.request
    from typing import List, Dict, Tuple, Optional
    
    # Check NumPy first
    import numpy as np
    print(f"✅ NumPy: {np.__version__}")
    
    # Import packages that commonly have NumPy 2.x issues
    try:
        import cv2
        print(f"✅ OpenCV: {cv2.__version__}")
    except ImportError as e:
        print(f"❌ OpenCV import failed: {e}")
        print("Run: !pip install --upgrade --force-reinstall opencv-python")
        raise
    
    try:
        import matplotlib.pyplot as plt
        print("✅ Matplotlib imported successfully")
    except ImportError as e:
        print(f"❌ Matplotlib import failed: {e}")
        print("This is likely a NumPy 2.x compatibility issue.")
        print("Please run the 'Fix Matplotlib NumPy 2.x Issue' cell above.")
        raise
    
    try:
        import torch
        print(f"✅ PyTorch: {torch.__version__}")
    except ImportError as e:
        print(f"❌ PyTorch import failed: {e}")
        print("Run: !pip install torch>=2.1.0 torchvision>=0.16.0")
        raise
    
    from PIL import Image
    print("✅ PIL imported successfully")
    
    try:
        import easyocr
        print("✅ EasyOCR imported successfully")
    except ImportError as e:
        print(f"❌ EasyOCR import failed: {e}")
        print("This is likely a NumPy compatibility issue.")
        print("Run: !pip install --upgrade --force-reinstall easyocr")
        raise
    
    # SAM imports
    try:
        from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator
        print("✅ SAM imported successfully")
    except ImportError as e:
        print(f"❌ SAM import failed: {e}")
        print("Run: !pip install git+https://github.com/facebookresearch/segment-anything.git")
        raise
    
    print("\n🎉 All libraries imported successfully with NumPy 2.x!")
    
except Exception as e:
    print(f"\n❌ Import failed: {e}")
    print("\n🔧 Troubleshooting steps:")
    print("1. Restart kernel (Kernel → Restart)")
    print("2. Run the dependency installation cell again")
    print("3. If matplotlib fails, run the matplotlib fix cell")
    print("4. Try alternative installation methods if needed")
    raise


In [ ]:
## 3. Download SAM Model


In [ ]:
# Download SAM model checkpoint
# Use /tmp/ directory for AWS SageMaker compatibility
models_dir = "/tmp/models" if "/opt/ml" in os.getcwd() else "models"
os.makedirs(models_dir, exist_ok=True)

# Download SAM ViT-B checkpoint (358MB)
checkpoint_url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"
checkpoint_path = os.path.join(models_dir, "sam_vit_b_01ec64.pth")

print(f"Model will be saved to: {checkpoint_path}")

if not os.path.exists(checkpoint_path):
    print("Downloading SAM model checkpoint (358MB)...")
    print("This may take a few minutes on first run...")
    try:
        urllib.request.urlretrieve(checkpoint_url, checkpoint_path)
        print("✅ Download complete!")
    except Exception as e:
        print(f"❌ Download failed: {e}")
        print("Please check your internet connection and try again.")
        raise
else:
    print("✅ SAM model checkpoint already exists.")


In [ ]:
## 4. Initialize SAM Model


In [ ]:
def load_sam_model(checkpoint_path: str):
    """Load SAM model"""
    print(f"Loading SAM model from {checkpoint_path}")
    
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"SAM checkpoint not found at {checkpoint_path}")
    
    # Load SAM model
    sam = sam_model_registry["vit_b"](checkpoint=checkpoint_path)
    
    # Use CPU for compatibility
    device = "cpu"
    sam.to(device=device)
    
    predictor = SamPredictor(sam)
    print(f"SAM loaded successfully on {device}")
    
    return sam, predictor

# Load SAM model
sam_model, sam_predictor = load_sam_model(checkpoint_path)


In [ ]:
def detect_ruler_and_calibrate(image, ocr_reader):
    """Detect ruler in image and calculate pixel-to-mm conversion ratio"""
    
    def extract_numeric_value(text):
        """Extract numeric value from OCR text"""
        import re
        cleaned = re.sub(r'[^\d.]', '', text)
        if cleaned:
            try:
                return float(cleaned)
            except ValueError:
                pass
        return None
    
    def preprocess_for_ocr(gray_roi):
        """Apply multiple preprocessing methods for better OCR"""
        variants = []
        
        # Original
        variants.append((gray_roi.copy(), "original"))
        
        # Adaptive threshold
        adaptive = cv2.adaptiveThreshold(gray_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY, 11, 2)
        variants.append((adaptive, "adaptive"))
        
        # Otsu threshold
        _, otsu = cv2.threshold(gray_roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        variants.append((otsu, "otsu"))
        
        # Inverted adaptive
        adaptive_inv = cv2.adaptiveThreshold(gray_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                           cv2.THRESH_BINARY_INV, 11, 2)
        variants.append((adaptive_inv, "adaptive_inv"))
        
        return variants
    
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Find ruler regions using edge detection
        edges = cv2.Canny(gray, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        ruler_regions = []
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < 500:
                continue
                
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = max(w, h) / min(w, h)
            
            # Rulers are typically long and narrow
            if aspect_ratio > 2.0 and max(w, h) > 100:
                ruler_regions.append((x, y, w, h, area))
        
        # Sort by area (larger regions first)
        ruler_regions.sort(key=lambda x: x[4], reverse=True)
        
        best_calibration = None
        best_confidence = 0
        
        # Try top ruler candidates
        for x, y, w, h, area in ruler_regions[:3]:
            roi = image[y:y+h, x:x+w]
            gray_roi = gray[y:y+h, x:x+w]
            
            markings = []
            
            # Try different preprocessing and orientations
            variants = preprocess_for_ocr(gray_roi)
            
            for processed_img, desc in variants:
                # Try original and rotated orientations
                for rotation, rot_desc in [(processed_img, "normal"), 
                                         (cv2.rotate(processed_img, cv2.ROTATE_90_CLOCKWISE), "rot90")]:
                    
                    # OCR with EasyOCR
                    try:
                        results = ocr_reader.readtext(rotation, paragraph=False)
                        
                        for (bbox, text, confidence) in results:
                            if confidence < 0.3:
                                continue
                                
                            numeric_value = extract_numeric_value(text)
                            if numeric_value is not None and 0 <= numeric_value <= 100:
                                # Calculate position
                                x_coords = [point[0] for point in bbox]
                                center_x = np.mean(x_coords)
                                
                                markings.append({
                                    'value': numeric_value,
                                    'confidence': confidence,
                                    'position': center_x,
                                    'text': text
                                })
                    except Exception as e:
                        continue
            
            # Calculate calibration from markings
            if len(markings) >= 2:
                markings.sort(key=lambda m: m['position'])
                
                ratios = []
                for i in range(len(markings) - 1):
                    mark1, mark2 = markings[i], markings[i + 1]
                    pixel_dist = abs(mark2['position'] - mark1['position'])
                    unit_dist = mark2['value'] - mark1['value']
                    
                    if unit_dist > 0 and pixel_dist > 10:
                        # Assume centimeters and convert to mm
                        px_per_mm = pixel_dist / (unit_dist * 10)
                        ratios.append(px_per_mm)
                
                if ratios:
                    avg_ratio = np.mean(ratios)
                    confidence = max(0.1, 1.0 - np.std(ratios) / avg_ratio) if len(ratios) > 1 else 0.7
                    
                    if confidence > best_confidence:
                        best_calibration = {
                            'px_per_mm': avg_ratio,
                            'confidence': confidence,
                            'markings_found': len(markings),
                            'method': 'ruler_detection',
                            'markings': [f"{m['value']:.0f}cm" for m in markings],
                            'ruler_region': (x, y, w, h),
                            'marking_positions': [(m['position'] + x, y + h//2, f"{m['value']:.0f}cm") for m in markings]
                        }
                        best_confidence = confidence
        
        if best_calibration:
            return best_calibration
        else:
            print("⚠️  Ruler detection failed, using default calibration")
            return {
                'px_per_mm': 30.0,
                'confidence': 0.3,
                'markings_found': 0,
                'method': 'default_fallback',
                'markings': [],
                'ruler_region': None,
                'marking_positions': []
            }
            
    except Exception as e:
        print(f"⚠️  Ruler detection error: {e}, using default calibration")
        return {
            'px_per_mm': 30.0,
            'confidence': 0.3,
            'markings_found': 0,
            'method': 'default_fallback',
            'markings': [],
            'ruler_region': None,
            'marking_positions': []
        }


In [ ]:
def draw_ruler_visualization(image, calibration):
    """Draw ruler region and markings on the visualization"""
    
    if calibration['ruler_region'] is None or calibration['markings_found'] == 0:
        return image
    
    vis_image = image.copy()
    x, y, w, h = calibration['ruler_region']
    
    # Draw ruler region with green rectangle
    cv2.rectangle(vis_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Add ruler region label
    ruler_label = f"Ruler Region ({calibration['method']})"
    cv2.putText(vis_image, ruler_label, (x, y-10), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    # Draw individual ruler markings
    for pos_x, pos_y, marking_text in calibration['marking_positions']:
        # Draw marking point
        cv2.circle(vis_image, (int(pos_x), int(pos_y)), 4, (0, 255, 255), -1)
        
        # Add marking text with background rectangle for visibility
        text_size = cv2.getTextSize(marking_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
        text_x = int(pos_x - text_size[0]//2)
        text_y = int(pos_y - 15)
        
        # Background rectangle
        cv2.rectangle(vis_image, (text_x-2, text_y-text_size[1]-2), 
                     (text_x+text_size[0]+2, text_y+2), (0, 0, 0), -1)
        
        # Text
        cv2.putText(vis_image, marking_text, (text_x, text_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
    
    # Add calibration info
    calib_info = f"Calibration: {calibration['px_per_mm']:.2f} px/mm"
    confidence_info = f"Confidence: {calibration['confidence']:.2f}"
    markings_info = f"Markings: {', '.join(calibration['markings'])}"
    
    # Draw calibration info with background
    y_offset = vis_image.shape[0] - 60
    
    for i, text in enumerate([calib_info, confidence_info, markings_info]):
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        text_y = y_offset + i * 20
        
        # Background rectangle
        cv2.rectangle(vis_image, (8, text_y - text_size[1] - 2), 
                     (text_size[0] + 12, text_y + 2), (0, 0, 0), -1)
        
        # Text
        cv2.putText(vis_image, text, (10, text_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    return vis_image


In [ ]:
### 🔧 Manual Update Required for Ruler Detection

**IMPORTANT:** To enable ruler detection with enhanced visualization, you need to manually update the `measure_objects_in_image()` function in the cell above.

**Step 1:** Replace this line:
```python
px_per_mm = 30.0  # This is a placeholder - you can implement ruler detection
```

**With this code:**
```python
# Ruler-based calibration
print("Performing ruler detection and calibration...")
calibration = detect_ruler_and_calibrate(resized_image, ocr_reader)
px_per_mm = calibration['px_per_mm']

print(f"Calibration: {px_per_mm:.3f} px/mm (method: {calibration['method']}, confidence: {calibration['confidence']:.2f})")
if calibration['markings_found'] > 0:
    print(f"Found {calibration['markings_found']} ruler markings: {calibration['markings']}")
```

**Step 2:** Replace the visualization section:
```python
# Create simple visualization
vis_image = resized_image.copy()
```

**With:**
```python
# Create enhanced visualization with ruler markings
vis_image = resized_image.copy()

# Add ruler visualization if detection was successful
if 'calibration' in locals():
    vis_image = draw_ruler_visualization(vis_image, calibration)
```

**Step 3:** Update the return statement to include full calibration data:
```python
'calibration': calibration  # instead of {'px_per_mm': px_per_mm, 'method': 'default'}
```

After making these changes, the system will display:
- **Green rectangle** around detected ruler region
- **Yellow circles** marking each detected ruler number
- **Number labels** with units (e.g., "3cm", "4cm", "5cm")
- **Calibration information** at the bottom of the image


In [ ]:
# EXAMPLE: Enhanced Measurement Function with Ruler Visualization
# Use this as reference to update the measurement function above

def measure_objects_in_image_enhanced(image_path: str, sam_model, sam_predictor, ocr_reader, display=True):
    """Complete measurement pipeline with ruler detection and enhanced visualization"""
    
    print(f"SAM MEASUREMENT SYSTEM: {image_path}")
    print("="*50)
    
    try:
        # Load and preprocess image
        original_image, resized_image, scale_x, scale_y = load_and_preprocess_image(image_path)
        
        # Detect yellow objects
        yellow_objects = detect_yellow_objects_hsv(resized_image)
        print(f"Detection found {len(yellow_objects)} yellow objects")
        
        if not yellow_objects:
            print("No yellow objects found")
            return None
        
        # Ruler-based calibration
        print("Performing ruler detection and calibration...")
        calibration = detect_ruler_and_calibrate(resized_image, ocr_reader)
        px_per_mm = calibration['px_per_mm']
        
        print(f"Calibration: {px_per_mm:.3f} px/mm (method: {calibration['method']}, confidence: {calibration['confidence']:.2f})")
        if calibration['markings_found'] > 0:
            print(f"Found {calibration['markings_found']} ruler markings: {calibration['markings']}")
        
        # Measure objects
        measurements = []
        for i, obj in enumerate(yellow_objects):
            # Get oriented bounding box
            rect = cv2.minAreaRect(obj['contour'])
            width_px, height_px = rect[1]
            
            # Ensure width > height
            if height_px > width_px:
                width_px, height_px = height_px, width_px
            
            # Convert to real world units
            width_mm = width_px / px_per_mm
            height_mm = height_px / px_per_mm
            
            measurement = {
                'object_id': i + 1,
                'width_mm': width_mm,
                'height_mm': height_mm,
                'width_inch': width_mm / 25.4,
                'height_inch': height_mm / 25.4,
                'width_px': width_px,
                'height_px': height_px,
                'area_px': obj['area']
            }
            measurements.append(measurement)
        
        print(f"\nSUCCESS!")
        
        for measurement in measurements:
            print(f"   Object {measurement['object_id']}: {measurement['width_mm']:.1f} x {measurement['height_mm']:.1f} mm")
            print(f"      ({measurement['width_inch']:.2f} x {measurement['height_inch']:.2f} inches)")
        
        # Create enhanced visualization with ruler markings
        vis_image = resized_image.copy()
        
        # Add ruler visualization first
        vis_image = draw_ruler_visualization(vis_image, calibration)
        
        # Draw yellow objects on top
        for i, obj in enumerate(yellow_objects):
            bbox = obj['bbox']
            x, y, w, h = bbox
            
            # Draw bounding box
            cv2.rectangle(vis_image, (x, y), (x+w, y+h), (0, 0, 255), 2)
            
            # Add measurement text with background for better visibility
            if i < len(measurements):
                measurement = measurements[i]
                text = f"Obj {measurement['object_id']}: {measurement['width_mm']:.1f}x{measurement['height_mm']:.1f}mm"
                
                # Calculate text size for background
                text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                
                # Background rectangle
                cv2.rectangle(vis_image, (x-2, y-text_size[1]-12), 
                             (x+text_size[0]+2, y-2), (0, 0, 0), -1)
                
                # Text
                cv2.putText(vis_image, text, (x, y-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # Add enhanced title
        title = 'Object Measurement with Ruler Detection'
        title_size = cv2.getTextSize(title, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
        cv2.rectangle(vis_image, (8, 8), (title_size[0]+12, 40), (0, 0, 0), -1)
        cv2.putText(vis_image, title, (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        if display:
            plt.figure(figsize=(15, 10))
            plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
            plt.title(f"Enhanced Measurement Results - {os.path.basename(image_path)}")
            plt.axis('off')
            plt.show()
        
        return {
            'success': True,
            'yellow_objects': yellow_objects,
            'measurements': measurements,
            'visualization': vis_image,
            'original_image': original_image,
            'resized_image': resized_image,
            'calibration': calibration
        }
        
    except Exception as e:
        print(f"Error: {e}")
        return {'success': False, 'error': str(e)}

print("Enhanced measurement function example created!")
print("Copy the relevant parts to update the main measurement function above.")


In [ ]:
def detect_ruler_and_calibrate(image, ocr_reader):
    """Detect ruler in image and calculate pixel-to-mm conversion ratio"""
    
    def extract_numeric_value(text):
        """Extract numeric value from OCR text"""
        import re
        cleaned = re.sub(r'[^\d.]', '', text)
        if cleaned:
            try:
                return float(cleaned)
            except ValueError:
                pass
        return None
    
    def preprocess_for_ocr(gray_roi):
        """Apply multiple preprocessing methods for better OCR"""
        variants = []
        
        # Original
        variants.append((gray_roi.copy(), "original"))
        
        # Adaptive threshold
        adaptive = cv2.adaptiveThreshold(gray_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY, 11, 2)
        variants.append((adaptive, "adaptive"))
        
        # Otsu threshold
        _, otsu = cv2.threshold(gray_roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        variants.append((otsu, "otsu"))
        
        # Inverted adaptive
        adaptive_inv = cv2.adaptiveThreshold(gray_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                           cv2.THRESH_BINARY_INV, 11, 2)
        variants.append((adaptive_inv, "adaptive_inv"))
        
        return variants
    
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Find ruler regions using edge detection
        edges = cv2.Canny(gray, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        ruler_regions = []
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < 500:
                continue
                
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = max(w, h) / min(w, h)
            
            # Rulers are typically long and narrow
            if aspect_ratio > 2.0 and max(w, h) > 100:
                ruler_regions.append((x, y, w, h, area))
        
        # Sort by area (larger regions first)
        ruler_regions.sort(key=lambda x: x[4], reverse=True)
        
        best_calibration = None
        best_confidence = 0
        
        # Try top ruler candidates
        for x, y, w, h, area in ruler_regions[:3]:
            roi = image[y:y+h, x:x+w]
            gray_roi = gray[y:y+h, x:x+w]
            
            markings = []
            
            # Try different preprocessing and orientations
            variants = preprocess_for_ocr(gray_roi)
            
            for processed_img, desc in variants:
                # Try original and rotated orientations
                for rotation, rot_desc in [(processed_img, "normal"), 
                                         (cv2.rotate(processed_img, cv2.ROTATE_90_CLOCKWISE), "rot90")]:
                    
                    # OCR with EasyOCR
                    try:
                        results = ocr_reader.readtext(rotation, paragraph=False)
                        
                        for (bbox, text, confidence) in results:
                            if confidence < 0.3:
                                continue
                                
                            numeric_value = extract_numeric_value(text)
                            if numeric_value is not None and 0 <= numeric_value <= 100:
                                # Calculate position
                                x_coords = [point[0] for point in bbox]
                                center_x = np.mean(x_coords)
                                
                                markings.append({
                                    'value': numeric_value,
                                    'confidence': confidence,
                                    'position': center_x,
                                    'text': text
                                })
                    except Exception as e:
                        continue
            
            # Calculate calibration from markings
            if len(markings) >= 2:
                markings.sort(key=lambda m: m['position'])
                
                ratios = []
                for i in range(len(markings) - 1):
                    mark1, mark2 = markings[i], markings[i + 1]
                    pixel_dist = abs(mark2['position'] - mark1['position'])
                    unit_dist = mark2['value'] - mark1['value']
                    
                    if unit_dist > 0 and pixel_dist > 10:
                        # Assume centimeters and convert to mm
                        px_per_mm = pixel_dist / (unit_dist * 10)
                        ratios.append(px_per_mm)
                
                if ratios:
                    avg_ratio = np.mean(ratios)
                    confidence = max(0.1, 1.0 - np.std(ratios) / avg_ratio) if len(ratios) > 1 else 0.7
                    
                    if confidence > best_confidence:
                        best_calibration = {
                            'px_per_mm': avg_ratio,
                            'confidence': confidence,
                            'markings_found': len(markings),
                            'method': 'ruler_detection',
                            'markings': [f"{m['value']}{m.get('unit', 'cm')}" for m in markings]
                        }
                        best_confidence = confidence
        
        if best_calibration:
            return best_calibration
        else:
            print("⚠️  Ruler detection failed, using default calibration")
            return {
                'px_per_mm': 30.0,
                'confidence': 0.3,
                'markings_found': 0,
                'method': 'default_fallback',
                'markings': []
            }
            
    except Exception as e:
        print(f"⚠️  Ruler detection error: {e}, using default calibration")
        return {
            'px_per_mm': 30.0,
            'confidence': 0.3,
            'markings_found': 0,
            'method': 'default_fallback',
            'markings': []
        }


In [ ]:
## 5. Initialize OCR Reader


In [ ]:
# Initialize OCR reader
print("Initializing OCR reader...")
ocr_reader = easyocr.Reader(['en'])
print("OCR reader initialized!")


In [ ]:
## 6. Core Functions

### Image Processing Functions


In [ ]:
def resize_image(image: np.ndarray, max_size: int = 800) -> np.ndarray:
    """Resize image for SAM processing"""
    h, w = image.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        new_w, new_h = int(w * scale), int(h * scale)
        image = cv2.resize(image, (new_w, new_h))
    return image

def load_and_preprocess_image(image_path: str):
    """Load and preprocess image"""
    # Load image
    original_image = cv2.imread(image_path)
    if original_image is None:
        raise ValueError(f"Could not load image from {image_path}")
    
    print(f"Original image: {original_image.shape[1]}x{original_image.shape[0]}")
    
    # Resize for processing
    resized_image = resize_image(original_image)
    print(f"Resized image: {resized_image.shape[1]}x{resized_image.shape[0]}")
    
    # Calculate scale factors
    scale_x = resized_image.shape[1] / original_image.shape[1]
    scale_y = resized_image.shape[0] / original_image.shape[0]
    
    return original_image, resized_image, scale_x, scale_y


In [ ]:
### Yellow Object Detection


In [ ]:
def detect_yellow_objects_hsv(image: np.ndarray) -> List[Dict]:
    """Yellow object detection using HSV color space"""
    print("\n=== YELLOW OBJECT DETECTION ===")
    
    # Convert to HSV for better color detection
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define yellow color range in HSV
    lower_yellow = np.array([15, 50, 50])
    upper_yellow = np.array([35, 255, 255])
    
    # Create binary mask for yellow regions
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # Morphological operations for noise reduction
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print(f"  Found {len(contours)} contours")
    
    # Process contours with filtering
    candidates = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        # Filter by area
        if area < 1000:
            continue
        
        # Calculate bounding rectangle
        bbox = cv2.boundingRect(contour)
        x, y, w, h = bbox
        
        # Calculate aspect ratio
        aspect_ratio = max(w, h) / min(w, h)
        
        # Filter by aspect ratio
        if aspect_ratio > 5:
            continue
        
        # Calculate quality metrics
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        solidity = area / hull_area if hull_area > 0 else 0
        extent = area / (w * h)
        
        # Quality score
        quality_score = (solidity * 0.4 + extent * 0.4 + min(1.0, area / 5000) * 0.2)
        
        # Only accept high-quality detections
        if quality_score > 0.5:
            candidates.append({
                'contour': contour,
                'bbox': bbox,
                'area': area,
                'aspect_ratio': aspect_ratio,
                'solidity': solidity,
                'extent': extent,
                'quality_score': quality_score,
                'method': 'hsv'
            })
    
    print(f"  After filtering: {len(candidates)} candidates")
    
    # Sort by quality score
    candidates.sort(key=lambda x: x['quality_score'], reverse=True)
    
    # Remove overlapping detections
    final_objects = []
    for candidate in candidates:
        is_duplicate = False
        for existing in final_objects:
            if calculate_bbox_overlap(candidate['bbox'], existing['bbox']) > 0.3:
                is_duplicate = True
                break
        
        if not is_duplicate:
            final_objects.append(candidate)
    
    print(f"  Final objects: {len(final_objects)}")
    for i, obj in enumerate(final_objects):
        print(f"    Object {i+1}: area={obj['area']}, quality={obj['quality_score']:.2f}")
    
    return final_objects

def calculate_bbox_overlap(bbox1: Tuple[int, int, int, int], bbox2: Tuple[int, int, int, int]) -> float:
    """Calculate overlap ratio between two bounding boxes"""
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    
    # Calculate intersection
    left = max(x1, x2)
    top = max(y1, y2)
    right = min(x1 + w1, x2 + w2)
    bottom = min(y1 + h1, y2 + h2)
    
    if left < right and top < bottom:
        intersection = (right - left) * (bottom - top)
        area1 = w1 * h1
        area2 = w2 * h2
        union = area1 + area2 - intersection
        return intersection / union if union > 0 else 0
    
    return 0


In [ ]:
### Main Measurement Function


In [ ]:
def measure_objects_in_image(image_path: str, sam_model, sam_predictor, ocr_reader, display=True):
    """Complete measurement pipeline - functional approach"""
    
    print(f"SAM MEASUREMENT SYSTEM: {image_path}")
    print("="*50)
    
    try:
        # Load and preprocess image
        original_image, resized_image, scale_x, scale_y = load_and_preprocess_image(image_path)
        
        # Detect yellow objects
        yellow_objects = detect_yellow_objects_hsv(resized_image)
        print(f"Detection found {len(yellow_objects)} yellow objects")
        
        if not yellow_objects:
            print("No yellow objects found")
            return None
        
        # Simple calibration (assume 30 pixels per mm as default)
        # In a real scenario, you would implement ruler detection here
        px_per_mm = 30.0  # This is a placeholder - you can implement ruler detection
        
        # Measure objects
        measurements = []
        for i, obj in enumerate(yellow_objects):
            # Get oriented bounding box
            rect = cv2.minAreaRect(obj['contour'])
            width_px, height_px = rect[1]
            
            # Ensure width > height
            if height_px > width_px:
                width_px, height_px = height_px, width_px
            
            # Convert to real world units
            width_mm = width_px / px_per_mm
            height_mm = height_px / px_per_mm
            
            measurement = {
                'object_id': i + 1,
                'width_mm': width_mm,
                'height_mm': height_mm,
                'width_inch': width_mm / 25.4,
                'height_inch': height_mm / 25.4,
                'width_px': width_px,
                'height_px': height_px,
                'area_px': obj['area']
            }
            measurements.append(measurement)
        
        print(f"\nSUCCESS!")
        
        for measurement in measurements:
            print(f"   Object {measurement['object_id']}: {measurement['width_mm']:.1f} x {measurement['height_mm']:.1f} mm")
            print(f"      ({measurement['width_inch']:.2f} x {measurement['height_inch']:.2f} inches)")
        
        # Create simple visualization
        vis_image = resized_image.copy()
        
        # Draw yellow objects
        for i, obj in enumerate(yellow_objects):
            bbox = obj['bbox']
            x, y, w, h = bbox
            
            # Draw bounding box
            cv2.rectangle(vis_image, (x, y), (x+w, y+h), (0, 0, 255), 2)
            
            # Add measurement text
            if i < len(measurements):
                measurement = measurements[i]
                text = f"Obj {measurement['object_id']}: {measurement['width_mm']:.1f}x{measurement['height_mm']:.1f}mm"
                cv2.putText(vis_image, text, (x, y-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # Add title
        cv2.putText(vis_image, 'Object Measurement Results', (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        if display:
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
            plt.title(f"Measurement Results - {os.path.basename(image_path)}")
            plt.axis('off')
            plt.show()
        
        return {
            'success': True,
            'yellow_objects': yellow_objects,
            'measurements': measurements,
            'visualization': vis_image,
            'original_image': original_image,
            'resized_image': resized_image,
            'calibration': {'px_per_mm': px_per_mm, 'method': 'default'}
        }
        
    except Exception as e:
        print(f"Error: {e}")
        return {'success': False, 'error': str(e)}


In [ ]:
## 7. Usage Instructions

### Step 1: Upload Your Images
Upload your test images to the SageMaker environment. Create a folder called 'test_images' and place your images there.


In [ ]:
# Create directories for test images and outputs
os.makedirs('test_images', exist_ok=True)
os.makedirs('output', exist_ok=True)

print("Directories created. Please upload your test images to the 'test_images' folder.")
print("You can use the file browser on the left to upload images.")


In [ ]:
### Step 2: Process Your Images


In [ ]:
# Process all images in the test_images folder
test_images = []
if os.path.exists('test_images'):
    for filename in os.listdir('test_images'):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            test_images.append(os.path.join('test_images', filename))

print(f"Found {len(test_images)} test images:")
for img in test_images:
    print(f"  - {img}")

if not test_images:
    print("No test images found. Please upload images to the 'test_images' folder.")
    print("Supported formats: .jpg, .jpeg, .png, .bmp")

# Process each image
results = []
for image_path in test_images:
    print(f"\n{'='*60}")
    print(f"PROCESSING: {image_path}")
    print(f"{'='*60}")
    
    # Run measurement system
    result = measure_objects_in_image(image_path, sam_model, sam_predictor, ocr_reader, display=True)
    
    if result and result['success']:
        results.append(result)
        
        # Save visualization
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        output_path = f'output/{base_name}_measurement_result.jpg'
        cv2.imwrite(output_path, result['visualization'])
        print(f"\nSaved visualization: {output_path}")
    else:
        print(f"\nFailed to process {image_path}")

print(f"\n\nProcessing complete! Successfully processed {len(results)} images.")


In [ ]:
### Step 3: Process Individual Images

Use this cell to process a single specific image:


In [ ]:
# Process a single image
# Replace 'your_image_path.jpg' with the actual path to your image
single_image_path = 'test_images/your_image_path.jpg'

if os.path.exists(single_image_path):
    result = measure_objects_in_image(single_image_path, sam_model, sam_predictor, ocr_reader, display=True)
    
    if result and result['success']:
        print("\nProcessing successful!")
        
        # Save result
        base_name = os.path.splitext(os.path.basename(single_image_path))[0]
        output_path = f'output/{base_name}_detailed_result.jpg'
        cv2.imwrite(output_path, result['visualization'])
        print(f"\nSaved detailed result: {output_path}")
    else:
        print("Processing failed!")
else:
    print(f"Image not found: {single_image_path}")
    print("Please update the path to point to your image file.")


In [ ]:
## 8. Summary and Results

### View Output Files


In [ ]:
# List all output files
output_files = []
if os.path.exists('output'):
    for filename in os.listdir('output'):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            output_files.append(os.path.join('output', filename))

print(f"Generated {len(output_files)} output files:")
for file in output_files:
    file_size = os.path.getsize(file) / 1024  # Size in KB
    print(f"  - {file} ({file_size:.1f} KB)")

print("\nYou can download these files using the file browser on the left.")

# Create summary of results
if 'results' in locals() and results:
    print("\n" + "="*60)
    print("MEASUREMENT SUMMARY")
    print("="*60)
    
    for i, result in enumerate(results):
        print(f"\nImage {i+1}:")
        print(f"  Objects detected: {len(result['yellow_objects'])}")
        print(f"  Calibration: {result['calibration']['px_per_mm']:.2f} px/mm")
        
        print("  Measurements:")
        for measurement in result['measurements']:
            print(f"    Object {measurement['object_id']}: {measurement['width_mm']:.1f} x {measurement['height_mm']:.1f} mm")
            print(f"      ({measurement['width_inch']:.2f} x {measurement['height_inch']:.2f} inches)")
else:
    print("\nNo results to summarize. Please run the processing cells first.")


In [ ]:
## 9. Notes and Instructions

This notebook provides a complete standalone object size estimation system that:

### Key Features:
- **Functional programming approach** - No classes, just functions
- **AWS SageMaker compatible** - Runs entirely in the notebook environment
- **SAM-based segmentation** - Uses Facebook's Segment Anything Model
- **Yellow object detection** - HSV color space filtering
- **Simple calibration** - Default px/mm ratio (can be enhanced with ruler detection)
- **Visual results** - Clear annotations and measurements

### Usage Instructions:
1. **Run all cells in sequence** - Start from the top and execute each cell
2. **Upload your images** - Place test images in the 'test_images' folder
3. **Process images** - Run the processing cells to analyze your images
4. **Download results** - Get processed images from the 'output' folder

### Customization Options:
- **Adjust calibration**: Modify the `px_per_mm` value in the measurement function
- **Change color detection**: Modify HSV ranges for different colored objects
- **Add ruler detection**: Implement OCR-based ruler detection for better calibration
- **Enhance visualization**: Add more detailed annotations and measurements

### Supported Image Formats:
- JPEG (.jpg, .jpeg)
- PNG (.png)
- BMP (.bmp)

### System Requirements:
- Python 3.7+
- OpenCV
- NumPy
- Matplotlib
- PyTorch
- SAM (Segment Anything Model)
- EasyOCR

The system is designed to be robust and handle various image qualities and lighting conditions. For best results, ensure good lighting and clear visibility of yellow objects in your images.
